---
title: "Meta Learning in Neural Networks — A Survey"
date: 2025-11-13       
description: "메타 러닝 관련 논문 요약 및 주요 내용"
categories: [MetaLearning, Survey, Review]
author: "김한울"
---


# Paper Review: Meta Learning in Neural Networks: A Survey

``` 
@article{hospedales2021meta,
  title={Meta-learning in neural networks: A survey},
  author={Hospedales, Timothy and Antoniou, Antreas and Micaelli, Paul and Storkey, Amos},
  journal={IEEE transactions on pattern analysis and machine intelligence},
  volume={44},
  number={9},
  pages={5149--5169},
  year={2021},
  publisher={IEEE}
}
```

`업데이트 내역`

||||
|-|-|-|
|2025-11-14|태스크 분포 관점 업데이트||
|2025-11-15|Bilevel optimization View 업데이트||

## 초록

`Learning-to-learn`이라고도 불리는 메타-러닝 분야는 최근 몇 년간 관심이 급격히 증가해왔습니다. 
고정된 학습 알고리즘을 사용하여 처음부터 과제를 해결하는 기존의 AI 접근 방식과는 대조적으로, 
메타-러닝은 다수의 학습 에피소드 경험을 바탕으로 **학습 알고리즘 자체를 개선하는 것을 목표**로 합니다. 
이 패러다임은 데이터 및 계산 병목 현상, 그리고 일반화 성능 등 딥러닝의 여러 기존 난제들을 해결할 기회를 제공합니다. 
본 서베이는 현대 메타-러닝의 전반적인 동향을 기술합니다. 먼저 메타-러닝의 정의를 논하고, 전이 학습(transfer learning) 및 하이퍼파라미터 최적화(hyperparameter optimization)와 같은 관련 분야와 비교하여 그 위치를 정립합니다. 
다음으로, 오늘날의 메타-러닝 방법 공간을 더 포괄적으로 분석하는 새로운 분류 체계를 제안합니다. 
또한 퓨샷 러닝(few-shot learning) 및 강화 학습(reinforcement learning)과 같은 메타-러닝의 유망한 응용 분야와 성공 사례들을 살펴봅니다. 마지막으로, 아직 해결되지 않은 과제들과 향후 연구를 위한 유망한 영역들에 대해 논의합니다.

## Introduction

메타러닝이란, "머신러닝 모델"이 "공부하는 법"자체를 배우게 하는 새로운 패러다임이라고 할 수 있음

> Meta-learning provides an alternative paradigm where a machine learning model gains experience over multiple learning episodes – often covering a distribution of related tasks – and uses this experience to improve its future learning performance.

1. provides an alternative paradigm?

* 기존 방식?    
  기계에게 '고양이 사진 분류'라는 숙제 하나를 주고, 그 숙제만 잘 풀도록 처음부터 끝까지 가르침. 다른 숙제(예: '개 사진 분류')를 주면 또 처음부터 새로 배워야 함.   
* 대안(메타러닝):    
  기계에게 숙제 하나만 주는 게 아니라, **"스스로 공부하는 법"**을 터득하게 만들자!   

2. a machine learning model gains experience over multiple learning episodes

* 비유를 해보자면    
  사람에게 수학만 가르치는 게 아니라, 수학, 과학, 국어 등 **여러 과목의 문제(관련된 과제들)**를 풀게 하는 행위랑 비슷함.   
  이 과정에서 모델은 단순히 개별 문제를 푸는 법을 배우는 게 아니라, 문제들 사이의 공통점이나 문제 해결의 **'요령(패턴)'**을 깨닫게 됨. 이것이 바로 **"경험"**이라고 할 수 있음.
* `learning episodes`: 문제 하나하나를 풀어보는 경험 한 번 한 번을 의미.


보통의 경우에, 그리고 역사적으로 보면, 

1. 머신러닝   
+ 기계가 데이터를 잘 이해할 수 있도록 데이터의 핵심적인 특징(feature)을 사람 전문가(human-expert)가 직접 추출함.
  + hand-engineered features   
+ 그 후, 사람이 뽑아낸 특징 데이터를 입력으로 받아서, 기계가 정답을 맞히는 패턴을 학습.
  + 이미지 문제를 예로 들었을 때, 기계는 원본 이미지를 보고 있는 것이 아니라, 사람이 가공해서 준 특징 값들만 보게 됨.   
+ 즉, 머신러닝에서 **특징 추출 단계**와 **모델 학습 단계**는 분리 되어 있음.



2. 딥러닝   
+ 위 머신러닝의 두 단계인 **특징 추출 단계**와 **모델 학습 단계**를 하나의 단계로 통합해버림.    
  + 특징과 모델의 공동 학습 (Joint feature and model learning)    
+ 예전 처럼 특징을 정성스럽게 추출할 필요가 없음.
  + 원본 데이터(이미지 형태 그대로!) 모델에 그대로 입력하면,   
  + 모델이 내부의 여러 계층(layer)을 거치면서 자동으로 특징을 찾아내고, 동시에 그 특징을 이용해 분류하는 방법까지 한꺼번에 학습함.   
+ 고양이 사진 분류하는 모델을 예로 들어보면...   
  + 초반 계층: 이미지의 가장 기본적인 특징(선, 색상, 명암 대비 등)을 스스로 학습.   
  + 중간 계층: 초반 계층에서 학습한 기본 특징들을 조합하여 더 복잡한 특징(눈, 코, 귀의 형태 등)을 학습.   
  + 후반 계층: 중간 계층의 복잡한 특징들을 다시 조합하여 최종적으로 "이것이 고양이다"라고 판단하는 방법을 학습.
+ 어떤 특징이 중요한지를 기계가 데이터로부터 직접 배우는 것.    
+ 사람이 "귀가 중요해, 수염이 중요해"라고 알려줄 필요가 없게 됨. 
+ 이처럼 딥러닝에서는 특징을 배우는 과정과 그 특징으로 정답을 맞히는 과정이 '공동으로(jointly)', 즉 '동시에' 최적화 됨.   

3. 신경망에서의 메타러닝?
+ 특징 추출 단계, 모델 학습 단계, 그리고 `알고리즘`을 하나의 단계로 통합하고자 하는 학습 방식이라고 할 수 있음.


Thrun은 '러닝-투-런'을 다음과 같이 측정 가능하게(operationally) 정의하고 있습니다. 

> Thrun [7] operationally defines learning-to-learn as occurring when a learner’s performance at solving tasks drawn from a given task family improves with respect to the number of tasks seen.   
  > *해당 책은 한 권에 40만원이라 읽어보지는 못할 것 같다. S. Thrun and L. Pratt, “Learning To Learn: Introduction And Overview,” in Learning To Learn, 1998 *

> 학습자(AI)가 관련된 과제 그룹(task family)에서 여러 과제를 풀어볼수록,    
> 즉 더 많은 종류의 과제를 경험할수록 새로운 과제를 푸는 성능이 향상되는 현상.

+ task family (과제 그룹): 서로 다르지만 관련이 있는 문제들의 묶음입니다.   
  + 예시: ('고양이/개 분류', '사자/호랑이 분류', '새/물고기 분류')는 모두 **"동물 분류"**라는 하나의 task family에 속합니다.   
  + AI가 '고양이/개 분류' 문제를 푼 다음, '사자/호랑이 분류' 문제를 풀고, 또 '새/물고기 분류' 문제를 푸는 등... 이렇게 다양한 종류의 과제(tasks)를 더 많이 경험할수록, 나중에 처음 보는 '코끼리/기린 분류' 문제도 더 잘 풀게 된다는 것입니다. 
  + 즉, 경험하는 과제의 '종류'가 늘어날수록 똑똑해집니다.

> conventional machine learning performance improves as more data from a single task is seen

+ 기존 머신러닝은 하나의 과제(a single task)에 대한 데이터가 많아질수록 성능이 향상됩니다.

**공짜 점심은 없다(no free lunch) Theorem** 에 대항하는 알고리즘 ㅋㅋ

> This perspective [27]–[29] views meta-learning as a tool to manage the ‘no free lunch’ theorem [30] and improve generalization by searching for the algorithm (inductive bias) that is best suited to a given problem, or problem family.

> 이러한 관점은 메타러닝을 '공짜 점심은 없다'는 정리(한계)에 대처하는 도구로 봅니다. 즉, 주어진 문제나 문제 그룹에 가장 적합한 알고리즘(귀납적 편향)을 탐색함으로써 일반화 성능을 향상시키는 도구라는 것입니다.

> However, this definition can include transfer, multi-task, feature-selection, and model-ensemble learning, which are not typically considered as meta-learning today.

> 하지만, 이러한 정의는 전이 학습, 다중과제 학습, 특징 선택, 모델 앙상블 학습까지 포함할 수 있는데, 이들은 오늘날 일반적으로 메타러닝으로 간주되지 않습니다.

+ "문제에 맞는 해결책을 찾는다"는 정의가 너무 광범위해서, 관련은 있지만 엄연히 다른 여러 기법들까지 전부 '메타러닝'의 범주에 포함시켜 버린다는 문제가 있다는 맥락입니다. 이 관점은 메타러닝의 철학을 이해하는 데는 도움이 되지만, 현대의 메타러닝을 다른 기법들과 구분 짓기에는 그 정의가 너무 모호하고 포괄적이라는 한계가 있습니다. 오늘날의 메타러닝은 단순히 알고리즘을 '선택'하거나 '재사용'하는 것을 넘어, '학습하는 과정 자체'를 최적화하는 더 구체적인 의미로 사용되기 때문입니다.

이 논문이 Review하려고 하는 대상 논문?

+ 본 논문에서는 현대의 신경망 기반 메타러닝에 집중하고 있습니다. 
  + 참고문헌 [27], [28]에 따라 **'알고리즘 학습'**으로 간주하지만, 특히 이 학습이 명시적으로 정의된 목적 함수(예: 교차 엔트로피 손실)의 종단간(end-to-end) 학습을 통해 달성되는 경우에 초점을 맞춥니다. 
  + 이에 더해, 우리는 단일 과제 메타러닝을 고려하고, 강건성(robustness) 및 컴퓨팅 효율성과 같은 더 폭넓고 다양한 (메타) 목적 함수에 대해서도 논의할 것입니다.

   
+ 본 논문에서는 메타러닝의 방법론과 응용 분야를 모두 다루려고 함. 
  + 먼저, 이 분야의 연구들을 이해하고 그 위상을 정립하는 데 사용할 수 있는 고수준의 문제 정형화(formalization)를 통해 메타러닝을 소개합니다
  + 그런 다음, **메타-표현(meta-representation), 메타-목적(meta-objective), 메타-최적화기(meta-optimizer)**라는 관점에서 새로운 분류 체계를 제공할 것입니다. 
  + 이 프레임워크는 새로운 메타러닝 방법을 개발하고 다양한 응용에 맞게 맞춤화하기 위한 **설계 공간(design-space)**을 제시합니다. 
  + 우리는 퓨샷 학습, 강화 학습, 구조 탐색 등 여러 인기 있고 새롭게 부상하는 응용 분야를 살펴보고, 전이 학습 및 다중과제 학습과 같은 관련 주제와 비교하여 메타러닝의 위상을 정립할 것입니다. 
  + 마지막으로, 아직 해결되지 않은 중요한 과제들과 미래 연구를 위한 유망한 영역들을 논의하며 마무리하겠습니다.


## 배경

메타러닝은 현대 신경망 문헌 내에서조차 
다양하고 일관성 없는 방식으로 사용되어 왔기 때문에 "한 단어, 한 문장"으로 명확하게 정의하기가 어려운 상태임. 

+ 이 섹션에서는 우리의 정의와 주요 용어를 소개하고, 관련 분야와 비교하여 메타러닝의 position을 정립하고 있음.


#### 메타러닝?

**'학습하는 방법을 학습하는 것(learning to learn)'**

+ 이는 **여러 학습 에피소드에 걸쳐 학습 알고리즘을 개선하는 과정을 의미**함. 

+ 기존의 머신러닝이 여러 데이터 인스턴스에 걸쳐 모델 예측을 개선하는 것과 대조적. 


+ **기반 학습(base learning)** 중에는 **내부(inner)** (또는 하위/기반) 학습 알고리즘이 이미지 분류와 같은 과제를 해결하며, 이는 데이터셋과 목적 함수에 의해 정의됨. 흔히 inner loop라는 표현으로도 많이 쓰임    
+ **메타러닝** 중에는 **외부(outer)**(또는 상위/메타) 알고리즘이 내부 학습 알고리즘을 업데이트하여, 그것이 학습하는 모델이 외부 목적 함수를 개선하도록 만듦. 
  + 예를 들어, 이 외부 목적 함수는 내부 알고리즘의 일반화 성능이나 학습 속도가 될 수 있음. 
  + (기반 알고리즘, 학습된 모델, 성능) 튜플로 구성된 기반 과제의 학습 에피소드들은 외부 알고리즘이 기반 학습 알고리즘을 학습하는 데 필요한 인스턴스를 제공하는 것으로 볼 수 있습니다.

  > inner loop의 결과물이라고 할 수 있는 (Base Algorithm, Trained Model, Performance)을 outer loop의 알고리즘이 "학습 데이터"로 삼아서 "기반 알고리즘 자체를 좋게 만드는 방법"을 학습하게 됨.
 


> 메타 러닝은 여러 학습 에피소드에 걸쳐 학습 알고리즘을 개선하는 과정?

위에 정의된 대로라면, `Cross Validtion`을 통한 하이퍼파라미터의 무작위 탐색과 같은 많은 기존 알고리즘이 메타러닝의 정의에 포함되어 버림. 

#### 현대의 neural-network meta-learning의 두드러지는 특징

+ **명시적으로 정의된 메타 수준의 목적 함수**이 있고,    
+ 이 목적 함수에 대한 내부 알고리즘의 **end-to-end 최적화**가 이루어짐.


### 메타러닝을 공식화 해보자

### **기존 머신러닝** 

기존의 지도(supervised) 머신러닝에서는 (입력 이미지, 출력 레이블) 쌍과 같은 훈련 데이터셋 $D = \{(x_1, y_1), \dots, (x_N, y_N)\}$이 주어짐. 

$\theta$로 매개변수화된 예측 모델 $\hat{y} = f_{\theta}(x)$를 다음 식을 풀어 훈련시키게 됨:


$$
\theta^* = \arg\min_{\theta} \mathcal{L}(D; \theta, \omega) \quad (1) 
$$


* $\mathcal{L}$은 실제 레이블과 $f_{\theta}(\cdot)$가 예측한 값 사이의 오차를 측정하는 손실 함수. 
* $\omega$라는 조건이 걸려 있음.
  * 이는 학습하는 방법 $\omega$에 따라 이 식(1)의 해인 $\theta$가 달라질 수 있다는 의미임.   
  * $\omega$ 예를 들어보자면 Optimizer의 선택, model의 선택이 될 수 있음.   

* 일반화 성능은 알려진 레이블을 가진 여러 테스트 포인트를 평가하여 측정됨.



##### 기존 머신러닝의 2가지 가정

(1) 최적화 과정이 모든 문제 $D$에 대해 매번 처음부터 수행됨(from scratch)   
(2) 학습 방법 $\omega$는 사전에 지정됨.   


이때, $\omega$의 **specification**, 즉 `학습 방법을 어떻게 정하느냐`는 정확도나 데이터 효율성과 같은 성능 지표에 큰 영향을 미칠 수 있음. 

+ 메타러닝은 이러한 지표를 개선하기 위해 학습 알고리즘 자체를 *사전에 지정하고 고정하는 대신* 학습 알고리즘 자체를 학습하게 됨. 

+ Specification? 학습 알고리즘의 구체적인 내용과 구성이라고 할 수 있음

  > 최적화기(Optimizer)의 종류: SGD, Adam, RMSprop 등 어떤 것을 쓸 것인가?    
  > 학습률(Learning Rate): 학습률을 얼마로 설정할 것인가?   
  > 모델 구조(Model Architecture): 어떤 종류의 신경망(CNN, RNN 등)을 사용할 것인가?   
  > 정규화(Regularization) 방법: L1, L2, Dropout 등 어떤 정규화 기법을 적용할 것인가?   

### 과제 분포(Task-Distribution) 관점에서 본 메타러닝


> 메타러닝을 통해 여러 과제에 걸쳐 일반화할 수 있고,    
> 이상적으로는 새로운 과제를 접할 때마다 이전보다 더 잘 학습할 수 있게 해주는 **범용 학습 알고리즘**을 학습하자. 

**Notation**    

+ $p(\mathcal{T})$: 과제들의 분포
+ $\omega$: 어떤 학습 방법 
+ $\mathcal{T} = \{D, L\}$: 어떤 과제($\mathcal{T}$)는 데이터셋($D$)과 손실 함수($L$)의 조합이다. 
+ $D$: 데이터 셋
+ $\mathcal{L}(D, \omega)$: 데이터 셋 $D$에서 학습 방법 $\omega$를 사용해 훈련했을 때의 loss 값

위와 같이 정의 했을 때, '학습하는 법을 배우는 것'은 다음과 같이 표현할 수 있음.

$$
\min_{\omega} \mathbb{E}_{\mathcal{T} \sim p(\mathcal{T})} \mathcal{L}(D; \omega) \quad (2)
$$


'학습 방법', 즉 $\omega$는 **과제 전반의 지식(across-task knowledge)** 또는 **메타 지식(meta-knowledge)** 이라고 할 수 있음.


**식 (2)를 실제로 해결하려면?**

* 목표: 세상의 모든 과제 분포 $p(\mathcal{T})$에 대해 평균적으로 가장 좋은 성능을 내는 만능학습법 $\omega$를 찾자!   
* 현실: 모든 과제 분포의 모든 문제인 $p(\mathcal{T})$를 다룰 수는 없음.
* 타협: 그러니까, 우리가 가진 문제는 "전체 과제들의 분포 $p(\mathcal{T})$를 어느정도 대표할 수 있는 샘플들"이야! --> source tasks(소스 과제)

**Notation 2**

$\mathcal{D}_{\text{source}} = \{(D_{\text{source}}^{\text{train}}, D_{\text{source}}^{\text{val}})^{(i)}\}_{i=1}^M$

* **메타-훈련(meta-training)에 사용할 전체 데이터셋**을 나타내는 기호. 

하나씩 뜯어보면

+ $\mathcal{D}_{\text{source}}$ : **'소스 데이터셋(Source Dataset)'**이라는 뜻. 여기서 '소스(Source)'는 메타 지식(학습 노하우)을 배우는 **원천(Source)**이 된다는 의미. 즉, **"훈련용"**이라는 뜻.
+ $M$: 우리가 가지고 있는 **훈련용 과제(task)의 총 개수**. (예: 50개의 다른 종류의 동물 분류 문제)
+ $\{ \dots \}_{i=1}^M$: 괄호 안의 내용이 **1번부터 M번까지 M개**가 있다는 뜻.
+ $( \dots )^{(i)}$: 그중에서 **i번째 과제**를 의미. (예: 50개 중 17번째 과제)
+ $(D_{\text{source}}^{\text{train}}, D_{\text{source}}^{\text{val}})$: 하나의 과제($i$)가 두 종류의 데이터로 구성되어 있다는 뜻.
  + $D_{\text{train}}^{\text{source}}$: **'훈련용' 데이터(train data)**. 이 과제를 풀기 위해 공부하는 데 사용되는 데이터. 메타러닝에서는 이것을 특별히 **서포트셋(support set)**이라고 함.
  + $D_{\text{val}}^{\text{source}}$: **'검증용' 데이터(validation data)**. 위에서 공부한 내용으로 얼마나 잘하는지 쪽지시험을 보는 데 사용되는 데이터. 메타러닝에서는 이것을 특별히 **쿼리셋(query set)**이라고 함.

> "$\mathcal{D}_{\text{source}}$란, 총 M개의 훈련용 과제 묶음인데, 각 과제$i$는 '서포트셋(훈련용)'과 '쿼리셋(검증용)'이라는 두 개의 데이터 묶음으로 이루어져 있다."

아무튼, 저런 데이터 묶음으로 뭘할거냐 하면, 식 (3)을 풀기 위함이다.

$$
\omega^* = \arg\max_{\omega} \log p(\omega|\mathcal{D}_{\text{source}}) \quad (3)
$$

또, 식을 하나씩 요소별로 뜯어보자.

+ $\omega$ (오메가): 우리가 찾고 싶은 **'학습 방법' 또는 '공부법'**
+ $\omega^{\ast}$ (오메가 스타): 수많은 가능한 공부법($\omega$) 중에서 우리가 찾아낸 **최고의(optimal) 공부법**을 의미함.
+ $\arg\max_{\omega}$: "괄호 안의 값을 **최대(max)로 만드는** $\omega$를 **찾아라(arg)**"라는 명령어.
+ $p(\omega|\mathcal{D}_{\text{source}})$: **사후 확률(posterior probability)**.
  + **"우리가 가진 훈련용 과제 데이터($\mathcal{D}_{\text{source}}$)를 관찰했을 때, 어떤 공부법($\omega$)이 가장 그럴듯한가(정답일 확률이 높은가)?"**.

> "우리가 가진 **훈련용 과제 데이터($\mathcal{D}_{\text{source}}$)를 가장 잘 설명하고 해결할 수 있는, 가장 그럴듯한(확률이 가장 높은) 학습 방법($\omega$)을 찾아서, 그것을 우리의 최종 학습법($\omega^{\ast}$)으로 삼아라!"**

이제 메타-테스트 단계에서 사용되는 $Q$개의 타겟 과제(target tasks) 집합을 
$\mathcal{D}_{\text{target}} = \{(D_{\text{target}}^{\text{train}}, D_{\text{target}}^{\text{test}})^{(i)}\}_{i=1}^Q$
로 표기하며, 각 과제는 훈련 데이터와 테스트 데이터를 모두 가집니다. 메타-테스트 단계에서는 학습된 메타 지식 $\omega^*$를 사용하여 이전에 보지 못한 각 타겟 과제 $i$에 대한 기반 모델을 훈련합니다:

$$
\theta^{*(i)} = \arg\max_{\theta} \log p(\theta|\omega^*, D_{\text{target}}^{\text{train }(i)}) \quad (4)
$$

식 (1)의 기존 학습과 대조적으로, 타겟 과제 $i$의 훈련 세트에 대한 학습은 이제 사용할 알고리즘에 대한 메타 지식 $\omega^*$의 이점을 얻습니다. 이것은 좋은 초기 파라미터의 추정치[16]일 수도 있고, 전체 학습 모델[38] 또는 최적화 전략[39]일 수도 있습니다. 우리는 각 타겟 과제의 테스트 스플릿 $D_{\text{test}}^{\text{target}}(i)$에 대한 $\theta^{*(i)}$의 성능으로 메타 학습기의 정확도를 평가할 수 있습니다.

이러한 설정은 기존의 과소적합 및 과적합과 유사한 개념인 **메타-과소적합(meta-underfitting)** 과 **메타-과적합(meta-overfitting)** 으로 이어집니다. 특히, 메타-과적합은 소스 과제에서 학습된 메타 지식이 타겟 과제로 일반화되지 않는 문제입니다. 이는 비교적 흔하며, 특히 소수의 소스 과제만 사용할 수 있는 경우에 그렇습니다. 이것은 가설 공간 $\theta$를 소스 과제의 해법 주변으로 너무 강하게 제약하는 귀납적 편향 $\omega$를 학습하는 것으로 볼 수 있습니다.




**다음 섹션 넘어가기 전 수식 해설:** 

$\mathcal{D}_{\text{target}} = \{(D_{\text{target}}^{\text{train}}, D_{\text{target}}^{\text{test}})^{(i)}\}_{i=1}^Q$

이 수식은 **메타-테스팅(Meta-Testing) 단계**, 즉 최종 실력을 평가하는 데 사용되는 **"실전 시험 문제지 묶음"** 전체를 정의합니다.

이 구조를 세 단계로 나누어 이해하면 가장 쉽습니다.

#### 1단계: $\mathcal{D}_{\text{target}}$ - 전체 시험지 묶음

*   **이름**: 타겟 데이터셋 (The Target Dataset)
*   **역할**: 메타-훈련을 통해 학습된 '만능 공부법($\omega^*$)'이 얼마나 효과적인지 최종적으로 평가하기 위한 **전체 시험 문제들의 집합**입니다.
*   **표기 규칙**: `target`이 **아래 첨자(subscript)**로 붙어, 이 데이터셋 묶음의 **소속(Group)**이 '소스(훈련용)'가 아닌 '타겟(시험용)'임을 나타냅니다.

#### 2단계: $( \dots )^{(i)}$ 와 $\{ \dots \}_{i=1}^Q$ - i번째 시험지

*   **이름**: i번째 타겟 과제 (The i-th Target Task)
*   **역할**: 전체 시험지 묶음($\mathcal{D}_{\text{target}}$)은 총 **Q개의 개별 시험 문제(과제)**로 이루어져 있습니다. 이 표기는 그중 **i번째 시험 문제** 하나를 가리킵니다.
*   **비유**: '수능 시험'이라는 전체 묶음 속의 '수학 시험지' 하나에 해당합니다.

#### 3단계: $(D_{\text{target}}^{\text{train}}, D_{\text{target}}^{\text{test}})$ - 시험지 한 장의 구성

이것이 가장 중요한 부분입니다. '수학 시험지' 한 장은 두 부분으로 구성됩니다.

##### 가. $D_{\text{target}}^{\text{train}}$: 시험지 속 "참고 예시" 또는 "<보기>"

*   **정확한 표기**: `target`은 **아래 첨자**, `train`은 **윗 첨자**.
*   **역할**:
    1.  이 데이터는 이미 학습이 끝난 **'만능 공부법($\omega^*$)'을 개선하는 데 사용되지 않습니다.**
    2.  대신, $i$번째 새로운 문제를 만난 모델이 이 데이터를 **딱 몇 번만 보고** "아, 이 문제는 이런 유형이구나!"하고 **빠르게 적응(adaptation)**하는 데 사용됩니다.
    3.  이 적응 과정을 통해 $i$번째 문제에만 특화된 모델($\theta^{*(i)}$)이 만들어집니다.
*   **비유**: 시험 문제에 나오는 **`<보기>` 자료**와 같습니다. `<보기>`를 읽고 문제의 의도를 파악하고 적응하는 것이지, `<보기>`를 읽는다고 해서 근본적인 국어 실력($\omega$)이 오르는 것은 아닙니다.

##### 나. $D_{\text{target}}^{\text{test}}$: 시험지 속 "실제 채점 문제"

*   **정확한 표기**: `target`은 **아래 첨자**, `test`은 **윗 첨자**.
*   **역할**:
    1.  위에서 `<보기>`($D_{\text{target}}^{\text{train}}$)를 보고 적응을 마친 모델($\theta^{*(i)}$)에게 이 문제를 풀게 합니다.
    2.  모델의 답과 정답을 비교하여 **최종 성능(정확도)을 채점**합니다. 이 점수가 바로 메타 학습기의 $i$번째 과제에 대한 최종 실력입니다.
*   **비유**: `<보기>`를 참고하여 풀어야 하는 **실제 문제 1번, 2번, 3번**에 해당합니다.

#### 최종 정리

| 표기법 | 소속 (아래 첨자) | 역할 (윗 첨자) | 비유 |
| :--- | :--- | :--- | :--- |
| $\mathcal{D}_{\text{target}}$ | **타겟(Target)** | (없음) | **수능 시험 전체** (최종 평가 목적) |
| $D_{\text{target}}^{\text{train}}$ | **타겟(Target)** | **훈련(Train)** | 시험지 속 **`<보기>`** (새로운 유형에 적응) |
| $D_{\text{target}}^{\text{test}}$ | **타겟(Target)** | **테스트(Test)** | 시험지 속 **채점 문제** (최종 성능 평가) |


### 이중 최적화(Bilevel Optimization) 관점에서 본 메타러닝

이전 섹션의 논의는 다중 과제 시나리오에서 메타러닝의 일반적인 흐름을 설명했지만, 식 (3)의 메타-훈련 단계를 어떻게 풀어야 하는지는 명시하지 않았습니다. 
이는 보통 메타-훈련 단계를 **이중 최적화(bilevel optimization)** 문제로 구성함으로써 수행됩니다. 이 그림은 최적화기 기반(optimizer-based) 방법론(섹션 3.1 참조)에만 정확하게 들어맞는다고 할 수 있지만, 메타러닝의 작동 방식을 더 보편적으로 시각화하는 데 도움이 됩니다. 
이중 최적화[40]는 계층적 최적화 문제로, 하나의 최적화가 다른 최적화를 제약 조건으로 포함하는 구조를 의미합니다[17], [41]. 

이 표기법을 사용하면, 메타-훈련은 다음과 같이 정형화될 수 있습니다:

$$
\omega^* = \arg\min_{\omega} \sum_{i=1}^{M} \mathcal{L}^{\text{meta}}(\theta^{*(i)}(\omega), \omega, D_{\text{source}}^{\text{val }(i)}) \quad (5)
$$

$$
\text{s.t. } \theta^{*(i)}(\omega) = \arg\min_{\theta} \mathcal{L}^{\text{task}}(\theta, \omega, D^{\text{train }(i)}_{\text{source}}) \quad (6)
$$

여기서 $\mathcal{L}^{\text{meta}}$
와 
$\mathcal{L}^{\text{task}}$
는 각각 **외부(outer) 및 내부(inner) 목적 함수**를 의미하며, 퓨샷 분류의 경우 교차 엔트로피와 같은 것입니다. 외부와 내부 수준 사이의 **리더-팔로워(leader-follower) 비대칭성**에 주목하십시오: *내부 수준 최적화인 식 (6)은 외부 수준에서 정의된 학습 전략 $\omega$에 따라 조건부로 결정되지만, 자신의 훈련 중에는 $\omega$를 변경할 수 없습니다.*

여기서 $\omega$는 비볼록(non-convex) 최적화에서의 초기 조건[16], 정규화 강도와 같은 하이퍼파라미터[17], 또는 최적화할 손실 함수 $\mathcal{L}^{\text{task}}$의 매개변수화[42]까지도 나타낼 수 있습니다. 

+ 섹션 4.1에서 $\omega$에 대한 선택의 공간을 자세히 논의합니다. 

외부 수준 최적화는 훈련 후 검증 세트에서 좋은 성능을 보이는 모델 $\theta^{*(i)}(\omega)$를 생성하도록 $\omega$를 학습합니다.

+ 섹션 4.2에서는 $\omega$를 최적화하는 방법을 자세히 논의합니다. 
+ 섹션 4.3에서는 $\mathcal{L}^{\text{meta}}$가 검증 성능, 학습 속도, 모델 강건성 등 무엇을 측정할 수 있는지 고려합니다.

마지막으로, 위의 정형화는 과제 분포라는 개념을 사용한다는 점에 주목합니다. 

+ 이는 메타러닝 문헌에서 일반적이지만, 메타러닝의 필수 조건은 아닙니다. 

더 공식적으로, 만약 단일 훈련 및 테스트 데이터셋($M=Q=1$)이 주어진다면, 우리는 훈련 세트를 분할하여 메타-훈련을 위한
$\mathcal{D}_{\text{source}} = (D^{\text{train}}_{\text{source}}, 
D^{\text{val}}_{\text{source}})$
와 메타-테스트를 위한 
$\mathcal{D}_{\text{target}} = (D^{\text{train}}_{\text{source}} \cup D^{\text{val}}_{\text{source}}, D^{\text{test}}_{\text{target}})$를 얻을 수 있습니다. 
우리는 여전히 여러 에피소드에 걸쳐 $\omega$를 학습하며, 메타-훈련 중에는 보통 다른 훈련-검증 분할이 사용됩니다.

**comment**

메타러닝의 작동 방식을 **"두 단계로 이루어진 최적화"**라는 틀로 설명합니다. 마치 회사에서 **팀장(외부 루프)**과 **팀원(내부 루프)**이 협업하는 것과 같습니다.

#### 1. 두 개의 최적화 문제: 내부 루프와 외부 루프

*   **내부 최적화 (식 6) - 팀원의 임무**:
    *   팀장은 **업무 가이드라인($\omega$)**을 줍니다. (예: "이런 방식으로 초기 모델을 설정해봐", "학습률은 0.01로 해")
    *   팀원은 주어진 가이드라인($\omega$)과 훈련 데이터($D^{\text{train}}$)를 가지고, **자신의 과제($\theta$)를 가장 잘 해결하기 위해 최선을 다합니다.**
    *   **중요한 점**: 팀원은 팀장이 준 가이드라인($\omega$)을 바꿀 수 없습니다. 그냥 따라야 합니다. 그 결과물이 바로 **최적의 모델($\theta^*$)**입니다.

*   **외부 최적화 (식 5) - 팀장의 임무**:
    *   팀장은 팀원이 과제를 수행한 결과물($\theta^*$)을 가져와서 **실전 테스트($D^{\text{val}}$)**를 해봅니다.
    *   테스트 결과(성능)를 보고, **"내가 처음에 줬던 가이드라인($\omega$)이 과연 최선이었을까?"**를 고민합니다.
    *   **목표**: 팀원의 최종 성과($\mathcal{L}^{\text{meta}}$)가 가장 좋아지도록, **최초의 가이드라인($\omega$) 자체를 수정하고 개선**합니다. 이것이 바로 팀장의 최적화, 즉 메타러닝입니다.

#### 2. $\omega$는 무엇인가? (팀장의 가이드라인)

팀장이 주는 가이드라인($\omega$)은 여러 형태일 수 있습니다.
*   **초기 조건**: "업무를 이 지점($\theta_0$)에서 시작하면 가장 빨리 끝낼 수 있을 거야." (MAML 방식)
*   **하이퍼파라미터**: "이 업무는 꼼꼼함(정규화)이 중요하니, 정규화 강도를 0.5로 설정해."
*   **손실 함수**: "이 업무의 목표는 단순히 정확도를 높이는 게 아니라, 안정성도 고려해야 해." 라며 목표 자체를 재정의해 줌.

#### 3. 꼭 여러 과제가 필요한가? No!

*   보통 메타러닝은 여러 팀(과제)의 성과를 보고 최고의 가이드라인을 찾지만, 꼭 그럴 필요는 없습니다.
*   **단 하나의 매우 중요한 프로젝트(single task)**가 있다면, 프로젝트를 여러 단계로 나누고 각 단계마다 팀원이 업무를 수행하게 한 뒤, 그 결과를 보고 팀장이 계속해서 가이드라인을 수정해주는 방식도 가능합니다. 이것이 **'단일 과제 메타러닝'**입니다.

### 피드-포워드 모델(Feed-Forward Model) 관점에서 본 메타러닝

앞서 살펴본 바와 같이, 식 (5)-(6)과 같은 명시적인 반복 최적화를 통하지 않고, 피드-포워드 방식으로 모델을 합성하는 여러 메타러닝 접근법이 있습니다. 
이들은 복잡도에 차이가 있지만, 이 접근법 계열을 이해하기 위해서는 식 (2)의 추상적인 목표를 구체화하여 선형 회귀 메타-훈련을 위한 간단한 예시[43]를 정의하는 것이 도움이 될 수 있습니다.

$$\min_{\omega} \underset{(\mathcal{D}^{tr}, \mathcal{D}^{val}) \in \mathcal{T}}{\mathbb{E}_{\mathcal{T} \sim p(\mathcal{T})}} \sum_{(\mathbf{x}, y) \in \mathcal{D}^{val}} \left[ (\mathbf{x}^T \mathbf{g}_{\omega}(\mathcal{D}^{tr}) - y)^2 \right] \quad (7)$$


여기서 우리는 과제 분포에 대해 메타-훈련을 수행합니다. 각 과제에 대해 훈련 세트와 검증 세트가 주어집니다. 

+ 훈련 세트 $D^{\text{tr}}$은 벡터 $g_\omega$로 임베딩[44]되며, 이 벡터는 검증 세트의 예시 $x$를 예측하기 위한 선형 회귀 가중치를 정의합니다. 
+ 식 (7)을 최적화하는 것은 함수 $g_\omega$가 훈련 세트를 가중치 벡터로 매핑하도록 훈련함으로써 '학습하는 법을 배우는 것'입니다.
+ 따라서 $g_\omega$는 $p(\mathcal{T})$에서 추출된 새로운 메타-테스트 과제 $^{\text{te}}$에 대해서도 좋은 해법을 제공해야 합니다. 이 계열의 방법들은 사용되는 예측 모델 $g$의 복잡성과 서포트셋이 어떻게 임베딩되는지(예: 풀링, CNN, RNN 사용)[44]에 따라 다양합니다. 

이러한 모델들은 **상각(amortized)**[45] 모델로도 알려져 있는데, 이는 새로운 과제를 학습하는 비용이 $g_\omega(\cdot)$를 통한 피드-포워드 연산 한 번으로 줄어들기 때문입니다. 반복 최적화에 드는 비용은 이미 $\omega$의 메타-훈련 중에 지불되었습니다.

**comments**

이 섹션은 이전의 '이중 최적화' 방식과는 완전히 다른, **매우 빠르고 효율적인 메타러닝 방식**을 설명합니다.

#### 이전 방식 (이중 최적화)의 문제점

*   새로운 문제가 주어질 때마다, 내부 루프에서 **느린 최적화 과정(경사 하강법 등)을 여러 번 반복**해야 합니다.
*   비유: 학생이 새로운 수학 문제를 만날 때마다, 공책에 여러 번 계산을 반복하며 풀어야 합니다.

#### 새로운 방식 (피드-포워드 모델)의 아이디어

*   "느린 반복 계산 과정을 없애버리고, 그냥 **문제를 척 보면 바로 답이 나오는 '만능 공식 생성기'($g_\omega$)**를 만들자!"
*   비유: 학생이 문제 유형과 주어진 숫자들을 '만능 공식 생성기'에 넣으면, 계산 과정 없이 바로 그 문제에 맞는 '최적의 공식'이 튀어나오고, 그 공식으로 답을 구합니다.

#### 수식 (7) 해설

$$\min_{\omega} \underset{(\mathcal{D}^{tr}, \mathcal{D}^{val}) \in \mathcal{T}}{\mathbb{E}_{\mathcal{T} \sim p(\mathcal{T})}} \sum_{(\mathbf{x}, y) \in \mathcal{D}^{val}} \left[ (\mathbf{x}^T \mathbf{g}_{\omega}(\mathcal{D}^{tr}) - y)^2 \right] \quad (7)$$

*   **$D^{\text{tr}}$ (훈련 세트)**: 학생에게 주어진 **'참고 예제'** 데이터입니다.
*   **$g_{\omega}(D^{\text{tr}})$**: 이것이 바로 **'만능 공식 생성기'**입니다. 이 함수($g$)는 참고 예제 데이터($D^{\text{tr}}$)를 입력으로 받아서, 이 문제를 푸는 데 필요한 **최적의 모델 파라미터(가중치)**를 **한 방에(피드-포워드 연산으로) 출력**합니다.
*   **$x^T g_{\omega}(D^{\text{tr}})$**: '만능 공식 생성기'가 만들어준 공식($g_{\omega}(D^{\text{tr}})$)을 가지고 실제 문제($x$)를 푸는 과정입니다.
*   **$( \dots - y)^2$**: 예측값과 실제 정답($y$) 사이의 오차입니다.
*   **전체 의미**: 여러 종류의 과제에 대해, **"주어진 참고 예제($D^{\text{tr}}$)를 보고 최적의 공식($g_{\omega}(D^{\text{tr}})$)을 한 번에 만들어내는 생성기($g_\omega$)를 훈련시켜라!** 이 생성기는 어떤 문제가 주어져도 항상 좋은 공식을 만들어내야 한다."

#### 상각(Amortized)이라는 용어의 의미

*   **'상각(Amortize)'**은 회계 용어로, 큰 비용을 여러 기간에 걸쳐 나누어 처리한다는 의미입니다.
*   메타러닝에서 이 용어는, **가장 비용이 많이 드는 '느린 반복 최적화' 과정을 메타-훈련 때 미리 다 해치워버리고($\omega$를 학습),** 정작 새로운 문제를 풀 때는 그 비용을 거의 치르지 않는다는 의미에서 사용됩니다.
*   **메타-훈련 (비용이 비쌈)**: 수많은 과제를 풀어보며 '만능 공식 생성기'($g_\omega$)를 만드는 데는 엄청난 시간과 계산이 필요합니다. (비용을 미리 지불)
*   **메타-테스트 (비용이 거의 0)**: 일단 생성기만 만들어지면, 새로운 문제는 그냥 함수에 데이터 한 번 넣는 것으로 끝나므로 거의 즉시 해결됩니다. (미리 지불한 비용의 혜택을 봄)

이 피드-포워드 방식은 특히 **새로운 문제에 대한 반응 속도가 매우 빨라야 하는 응용 분야**에 매우 유용합니다.